In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
from tensorflow import keras

In [2]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [4]:
def draw_landmarks(image, results):
    #mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION) # Draw face connections
    #mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [5]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    #mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                             #mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             #mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             #) 
    # Draw pose connections
    #mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
    #                         mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
    #                         mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
    #                         ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [6]:
def extract_keypoints(results):
    #pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    #face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([lh, rh])

In [7]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data') 

# Actions that we try to detect
actions = np.array(['i love you','hello', 'I', 'to', 'forgive', 'hill', 'meet', 'want', 'yes', 'go', 'drink', 'now'])

# Thirty videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length
sequence_length = 30

In [8]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [9]:
label_map = {label:num for num, label in enumerate(actions)}
label_map

{'i love you': 0,
 'hello': 1,
 'I': 2,
 'to': 3,
 'forgive': 4,
 'hill': 5,
 'meet': 6,
 'want': 7,
 'yes': 8,
 'go': 9,
 'drink': 10,
 'now': 11}

In [10]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [11]:
X = np.array(sequences)

In [12]:
y = to_categorical(labels).astype(int)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15)

In [29]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, SimpleRNN, GRU
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras import regularizers

In [30]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [31]:
model = Sequential()
model.add((SimpleRNN(64, return_sequences=True, activation='relu', input_shape=(30,126))))
model.add((SimpleRNN(128, return_sequences=True, activation='relu')))
model.add((SimpleRNN(64, return_sequences=False, activation='relu')))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu', kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),
    bias_regularizer=regularizers.l2(1e-4),
    activity_regularizer=regularizers.l2(1e-5)))
model.add(Dense(actions.shape[0], activation='softmax'))

In [32]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [33]:
model.fit(X_train, y_train, epochs=200, callbacks=[tb_callback])

Epoch 1/200
10/10 [==============================] - 7s 31ms/step - loss: 2.4567 - categorical_accuracy: 0.1438
Epoch 2/200
10/10 [==============================] - 0s 33ms/step - loss: 2.3169 - categorical_accuracy: 0.1961
Epoch 3/200
10/10 [==============================] - 0s 34ms/step - loss: 1.9719 - categorical_accuracy: 0.3464
Epoch 4/200
10/10 [==============================] - 0s 31ms/step - loss: 1.7399 - categorical_accuracy: 0.3889
Epoch 5/200
10/10 [==============================] - 0s 36ms/step - loss: 1.5394 - categorical_accuracy: 0.4837
Epoch 6/200
10/10 [==============================] - 0s 37ms/step - loss: 1.4236 - categorical_accuracy: 0.4804
Epoch 7/200
10/10 [==============================] - 0s 41ms/step - loss: 1.6553 - categorical_accuracy: 0.3954
Epoch 8/200
10/10 [==============================] - 0s 41ms/step - loss: 1.5174 - categorical_accuracy: 0.4216
Epoch 9/200
10/10 [==============================] - 0s 41ms/step - loss: 1.3080 - categorical_accuracy:

10/10 [==============================] - 0s 23ms/step - loss: 0.0308 - categorical_accuracy: 0.9967
Epoch 74/200
10/10 [==============================] - 0s 22ms/step - loss: 0.0314 - categorical_accuracy: 0.9967
Epoch 75/200
10/10 [==============================] - 0s 22ms/step - loss: 0.0315 - categorical_accuracy: 0.9967
Epoch 76/200
10/10 [==============================] - 0s 24ms/step - loss: 0.0294 - categorical_accuracy: 0.9967
Epoch 77/200
10/10 [==============================] - 0s 27ms/step - loss: 0.0285 - categorical_accuracy: 0.9967
Epoch 78/200
10/10 [==============================] - 0s 27ms/step - loss: 0.0263 - categorical_accuracy: 0.9967
Epoch 79/200
10/10 [==============================] - 0s 27ms/step - loss: 0.0320 - categorical_accuracy: 0.9967
Epoch 80/200
10/10 [==============================] - 0s 28ms/step - loss: 0.0355 - categorical_accuracy: 0.9967
Epoch 81/200
10/10 [==============================] - 0s 33ms/step - loss: 0.0296 - categorical_accuracy: 0.9

10/10 [==============================] - 0s 24ms/step - loss: 0.0123 - categorical_accuracy: 1.0000
Epoch 146/200
10/10 [==============================] - 0s 27ms/step - loss: 0.0122 - categorical_accuracy: 1.0000
Epoch 147/200
10/10 [==============================] - 0s 25ms/step - loss: 0.0121 - categorical_accuracy: 1.0000
Epoch 148/200
10/10 [==============================] - 0s 24ms/step - loss: 0.0121 - categorical_accuracy: 1.0000
Epoch 149/200
10/10 [==============================] - 0s 25ms/step - loss: 0.0120 - categorical_accuracy: 1.0000
Epoch 150/200
10/10 [==============================] - 0s 23ms/step - loss: 0.0120 - categorical_accuracy: 1.0000
Epoch 151/200
10/10 [==============================] - 0s 25ms/step - loss: 0.0119 - categorical_accuracy: 1.0000
Epoch 152/200
10/10 [==============================] - 0s 24ms/step - loss: 0.0119 - categorical_accuracy: 1.0000
Epoch 153/200
10/10 [==============================] - 0s 27ms/step - loss: 0.0118 - categorical_accur

In [34]:
res = model.predict(X_test)

In [35]:
res

array([[1.94730019e-04, 1.25136268e-07, 7.74927139e-06, 2.18358659e-03,
        2.75960502e-07, 4.74516401e-06, 9.97091889e-01, 1.81561390e-05,
        2.72872967e-05, 4.67471909e-05, 1.99331544e-06, 4.22770798e-04],
       [1.45844563e-08, 1.44953105e-10, 4.82539281e-06, 2.12219520e-03,
        7.22228464e-08, 2.23520988e-06, 8.58617227e-07, 9.97841954e-01,
        6.06294681e-10, 1.33005997e-05, 8.44538533e-07, 1.35114451e-05],
       [6.61708643e-09, 3.06716263e-10, 3.82101971e-05, 2.02136755e-04,
        2.19697398e-07, 5.69833219e-07, 2.64053419e-08, 9.99725282e-01,
        2.14919096e-10, 3.45274225e-06, 5.21788230e-08, 3.00304218e-05],
       [9.99740541e-01, 7.92224881e-08, 4.86474055e-05, 2.22571241e-08,
        1.75914963e-08, 5.56888875e-11, 7.24420602e-07, 1.04883053e-08,
        1.65727615e-04, 2.50693022e-09, 1.04806865e-08, 4.42760575e-05],
       [9.99822438e-01, 2.20371259e-08, 2.44676321e-05, 4.11224876e-09,
        4.20136148e-09, 9.74204051e-12, 2.27812308e-07, 1.24

In [36]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [37]:
yhat = model.predict(X_test)

In [38]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [39]:
multilabel_confusion_matrix(ytrue, yhat) 

array([[[49,  0],
        [ 0,  5]],

       [[42,  5],
        [ 2,  5]],

       [[50,  0],
        [ 1,  3]],

       [[50,  0],
        [ 1,  3]],

       [[52,  0],
        [ 0,  2]],

       [[45,  0],
        [ 0,  9]],

       [[52,  1],
        [ 0,  1]],

       [[49,  0],
        [ 0,  5]],

       [[52,  0],
        [ 1,  1]],

       [[50,  0],
        [ 0,  4]],

       [[46,  2],
        [ 3,  3]],

       [[49,  0],
        [ 0,  5]]], dtype=int64)

In [40]:
accuracy_score(ytrue, yhat)

0.8518518518518519

In [41]:
colors = [(245,117,16), (117,245,16), (16,117,245),(245,117,16), (117,245,16), (16,117,245),(117,245,16), 
          (16,117,245),( 245,117,16), (117,245,16), (16,117,245), ( 245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [43]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.85

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        cv2.waitKey(50)
        sequence = sequence[-30:]
        cv2.waitKey(50)
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilitiesqqqqqqqq
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

drink
<class 'mediapipe.python.solution_base.SolutionOutputs'>
drink
<class 'mediapipe.python.solution_base.SolutionOutputs'>
drink
<class 'mediapipe.python.solution_base.SolutionOutputs'>
drink
<class 'mediapipe.python.solution_base.SolutionOutputs'>
drink
<class 'mediapipe.python.solution_base.SolutionOutputs'>
drink
<class 'mediapipe.python.solution_base.SolutionOutputs'>
drink
<class 'mediapipe.python.solution_base.SolutionOutputs'>
drink
<class 'mediapipe.python.solution_base.SolutionOutputs'>
drink
<class 'mediapipe.python.solution_base.SolutionOutputs'>
drink
<class 'mediapipe.python.solution_base.SolutionOutputs'>
drink
<class 'mediapipe.python.solution_base.SolutionOutputs'>
drink
<class 'mediapipe.python.solution_base.SolutionOutputs'>
drink
<class 'mediapipe.python.solution_base.SolutionOutputs'>
drink
<class 'mediapipe.python.solution_base.SolutionOutputs'>
drink
<class 'mediapipe.python.solution_base.SolutionOutputs'>
drink
<class 'mediapipe.python.solution_base.SolutionOu

In [44]:
model.save('final_v2.h5') #['i love you','hello', 'I', 'to', 'forgive', 'hill', 'meet', 'want', 'yes', 'go', 'drink'
#, 'now']